In [1]:
import requests
from bs4 import BeautifulSoup
from pandas import to_datetime, DataFrame, concat

In [2]:
def get_day_news(url):

    day_df = DataFrame()
    
    for page in range(1, 50):

        response = requests.get(url + str(page))
        soup = BeautifulSoup(response.text, 'lxml')
        
        items = soup.find_all(class_="archive-page__item _news")
        
        if len(items) == 0:
            return day_df
        
        dates = [to_datetime(url[35:45]) for data in range(len(items))]
        titles = []
        texts = []
        links = []
        
        for item in items:

            titles.append(item.find('h3').get_text().strip())

            news_url = 'https://lenta.ru' + item.find(class_='card-full-news _archive').get('href')
            links.append(news_url)
            news_soup = BeautifulSoup(requests.get(news_url).text, 'lxml')

            texts.append(' '.join([paragraph.get_text().strip() for paragraph in news_soup.find_all('p')][1:]))

        day_df = concat([day_df, DataFrame({'date': dates, 'title': titles, 'full_descr': texts, 'link': links})])

In [3]:
from datetime import date, timedelta

start_date, end_date = date(2020, 1, 1), date(2020, 1, 31)
delta = end_date - start_date

dates = [str(start_date + timedelta(days=i))[:4] + '/'
                + str(start_date + timedelta(days=i))[5:7] + '/'
                + str(start_date + timedelta(days=i))[8:10] for i in range(delta.days + 1)]
dates

['2020/01/01',
 '2020/01/02',
 '2020/01/03',
 '2020/01/04',
 '2020/01/05',
 '2020/01/06',
 '2020/01/07',
 '2020/01/08',
 '2020/01/09',
 '2020/01/10',
 '2020/01/11',
 '2020/01/12',
 '2020/01/13',
 '2020/01/14',
 '2020/01/15',
 '2020/01/16',
 '2020/01/17',
 '2020/01/18',
 '2020/01/19',
 '2020/01/20',
 '2020/01/21',
 '2020/01/22',
 '2020/01/23',
 '2020/01/24',
 '2020/01/25',
 '2020/01/26',
 '2020/01/27',
 '2020/01/28',
 '2020/01/29',
 '2020/01/30',
 '2020/01/31']

In [4]:
from tqdm import tqdm

df = DataFrame()

for date in tqdm(dates):
    df = concat([df, get_day_news('https://lenta.ru/rubrics/economics/' + date + '/page/')])

100%|███████████████████████████████████████████| 31/31 [03:40<00:00,  7.13s/it]


In [5]:
df.shape

(597, 4)

In [6]:
df.head(15)

,date,title,full_descr,link
0,2020-01-01,В России повысили зарплаты,В России повышен минимальный размер оплаты тру...,https://lenta.ru/news/2020/01/01/mrot/
1,2020-01-01,В России подорожали алкогольные напитки и сига...,В России подорожали алкогольные напитки и сига...,https://lenta.ru/news/2020/01/01/expensive/
2,2020-01-01,В России повысили пенсии,С 1 января 2020 года в России повысили пенсии ...,https://lenta.ru/news/2020/01/01/pensions/
3,2020-01-01,Банковские переводы по номеру телефона стали п...,Центробанк начал взимать комиссию с банков за ...,https://lenta.ru/news/2020/01/01/comission/
4,2020-01-01,В России утвердили стандарты органической еды,В России вступил в силу закон об органической ...,https://lenta.ru/news/2020/01/01/org/
5,2020-01-01,Российские театры освободили от налогов,Российские театры и другие учреждения культуры...,https://lenta.ru/news/2020/01/01/theaters/
6,2020-01-01,Покупать в интернете за границей стало сложнее,Покупать в интернете за границей стало сложнее...,https://lenta.ru/news/2020/01/01/tariff/
7,2020-01-01,На Украине подорожал газ,Цена на газ для населения Украины выросла с 58...,https://lenta.ru/news/2020/01/01/gas/
8,2020-01-01,В России стало больше самозанятых,В России стало больше самозанятых — специальны...,https://lenta.ru/news/2020/01/01/self/
9,2020-01-01,Суд отменил штраф американской компании за нар...,Федеральный суд Северного округа штата Техас о...,https://lenta.ru/news/2020/01/01/exxx/


In [7]:
df.iloc[7]['date']

Timestamp('2020-01-01 00:00:00')

In [8]:
df.iloc[7]['title']

'На Украине подорожал газ'

In [9]:
df.iloc[7]['full_descr']

'Цена на газ для населения Украины выросла с 5878 гривен (15,4 тысячи рублей) до 6600 гривен (17,3 тысячи рублей) за тысячу кубометров. Об этом говорилось в сообщении украинского Минэнерго. Указанные цены не включают стоимость транспортировки топлива. С 1 января государство прекратило регулировать цены на газ для населения, которые прежде удерживало на определенном уровне, компенсируя поставщикам разницу с рыночными ценами. С 2020 года цена определяется, исходя из мировых цен на топливо с учетом стоимости транспортировки. Таково было требование Международного валютного фонда при предоставлении очередного кредитного транша. До конца отопительного сезона цена тысячи кубометров, продаваемого «Нафтогазом» на оптовом рынке, составит 5,5 тысячи гривен без учета НДС (20 процентов), или 6,6 тысячи гривен с учетом НДС.'

In [10]:
df.iloc[7]['link']

'https://lenta.ru/news/2020/01/01/gas/'